In [1]:
# get ic decay
from get_data import get_data,bars
from singletrader.processors.cs_processor import CsWinzorize,CsNeutrualize
from singletrader.shared.utility import save_pkl
from singletrader.factorlib import FactorEvaluation,summary_plot
import pandas as pd
import plotly.express as px
from plotly.figure_factory import create_table
import warnings
warnings.filterwarnings('ignore')
from singletrader.datasdk.qlib.base import MultiFactor
from singletrader.shared.utility import load_pkls
from singletrader.datasdk.sql.dataapi import get_index_cons
cs_neu = CsNeutrualize()


{"level": "INFO", "time": "2023-04-04 17:16:58,667", "timeMillis": 1680599818435,"module": "numexpr.utils", "method": "python-service", "addition": "[utils.py 148 _init_num_threads]", "message": "Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8."}
{"level": "INFO", "time": "2023-04-04 17:16:58,668", "timeMillis": 1680599818435,"module": "numexpr.utils", "method": "python-service", "addition": "[utils.py 160 _init_num_threads]", "message": "NumExpr defaulting to 8 threads."}
{"level": "INFO", "time": "2023-04-04 17:16:59,019", "timeMillis": 1680599818435,"module": "singletrader-service", "method": "python-service", "addition": "[__init__.py 26 init]", "message": "开始初始化路径..."}
[31080:MainThread](2023-04-04 17:16:59,699) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
{"level": "INFO", "time": "2023-04-04 17:16:59,699", "timeMillis": 1680599818435,"module": "qlib.Initialization", "method": "python-service", "addition": "[

In [2]:
fields = []
names = []

fields += ['$close','$open','$high','$low','$avg','$volume','$circulating_market_cap','$turnover_ratio','$money','1/$pe_ratio']
bars = ['close','open','high','low','avg','volume','circulating_market_cap','turnover_ratio','money','ep']
names += bars

In [3]:
# 获取数据
start_date = '2018-01-01'
end_date = '2023-03-31'
data = MultiFactor(name=names,field=fields,start_date=start_date,end_date=end_date)._data
index_cons = get_index_cons(start_date=start_date,end_date=end_date)
factor_data = load_pkls(r'D:\projects\singletrader_pro\samples\factor_mining\tamplate01\predict')
# factor_data = factor_data[index_cons.reindex(factor_data.index)['zz1000']>0]
factor_data.index = factor_data.index.set_names(['date','code'])
factor_data.name = 'lgb'
factor_data_neu = cs_neu(factor_data)
all_factor = pd.concat([factor_data,factor_data_neu],axis=1).dropna()
all_factor.columns = ['raw','neu']

{"level": "INFO", "time": "2023-04-04 17:17:01,743", "timeMillis": 1680599818435,"module": "root", "method": "python-service", "addition": "[base.py 75 _data]", "message": "开始生成因子数据..."}
{"level": "INFO", "time": "2023-04-04 17:17:46,945", "timeMillis": 1680599818435,"module": "root", "method": "python-service", "addition": "[base.py 75 _data]", "message": "开始生成因子数据..."}


In [4]:

fe = FactorEvaluation(bar_data=data,factor_data=all_factor)
fe.get_summary(groups=10,base='avg',holding_period=5,add_shift=1)


start_date:None / end_date:None


,raw,neu
ic.mean,0.0727,0.0664
ic.t-stats,13.7824,15.9266
AnnRet_Long,0.2298,0.1844
AnnRet_short,-0.4512,-0.4425
AnnRet,0.6810,0.6270
SR_Long,4.0145,4.0319
SR_Short,-5.4306,-5.7374
SR,6.1092,7.0939
TO_Long,0.1293,0.1358
TO_Short,0.1282,0.1279


In [5]:
report = fe.get_factor_detail_report(factor='raw',cost=0.00,universe=index_cons['zz1000'],holding_period=1,groups=10,base='avg',total=False,add_shift=1,excess_return=True)
summary_plot(report)

In [6]:
report = fe.get_factor_detail_report(factor='neu',holding_period=5,groups=10,cost=0.0015,base='avg',universe=index_cons['zz500'],total=True,add_shift=1,excess_return=True)
summary_plot(report)